# Tutorial 13-01 - Create a Script Tool

## Develop Your Script Logic

#### 1.  Revisit Chapter 10 Tutorial 2 - Multiprocessing

In Chapter 10 Tutorial 2 we developed a multiprocessing script for exporting a zipped file geodatabase containing a feature class with highways for a specific county.  The basic steps we went through were:
 - Pick a county.
 - Replace spaces with underscores.
 - Create a file geodatabase.
 - Create a feature class in that geodatabase containing the highways for the county.
 - Compress the file geodatabase into a .zip file.
 - Delete the temporary file geodatabase.
After we developed and tested that logic, we refactored that code and turned it into a function.  We can reuse that function here and focus on building a geoprocessing tool around it.

In [ ]:
def zip_county_highways(full_fc_path, output_folder, county):
    
    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    # define a path for the zip file
    zip_file_path = os.path.join(output_folder, f"{county} Highways.zip") 

    # zip the file geodatabase
    with zipfile.ZipFile(zip_file_path, "w") as zipper:
        for root, dirs, files in os.walk(fgdb[0]):
            for file in files:
                fpath = os.path.join(root, file)
                zpath = os.path.relpath(
                            os.path.join(root, file),
                            os.path.join(fgdb[0], '..')
                        )
                zipper.write(
                    fpath,
                    zpath
                )
    
    # delete the file geodatabase
    arcpy.management.Delete(fgdb)
    
    # return the zip file path
    return zip_file_path

#### 2.  Import packages

Now that you've got your script logic all set out with a function, you can execute that function with a few lines of code.  

First you'll need to import the necessary packages to execute that code.  In this case, you're using `arcpy` to create the file geodatabase and export the feature class.  You're using `os` to manage file and folder paths.  You're also using the `zipfile` package to compress your file geodatabase into a zip file.


In [ ]:
# import packages
import arcpy
import os
import zipfile

#### 3. Set up test inputs

Now you'll define your inputs.  You'll need an input feature class and an output folder for the zip files.

In [ ]:
# input file geodatabase path
full_fc_path = r"..\Chapter 03 - ArcPy Basics\Chapter 03 Files\Chapter 02 - Working with Maps.gdb\Highways_intersect"

# output folder
output_folder = r".\zipped_outputs"

#### 4.  Test your function.

After you've imported your packages and defined your inputs you can test your function.  It's usually good to make sure all your logic is sound before starting to create a script tool.

In [ ]:
# export a zip file for Butte County
zip_county_highways(full_fc_path, output_folder, "Butte County")

## Create a Script Tool

Now that you've got your script all set, you can work on setting up a user interface for your script.  Some of the work we do here is going to be scripting and some is going to be in the ArcGIS Pro user interface. 

#### 1.  Open ArcGIS Pro

If you aren't already using this notebook in ArcGIS Pro, go ahead and open the ArcGIS Pro Project included in this chapter's working files.

#### 2.  Create a new script tool

In the **Catalog View** of the included project, use the **Contents** pane to find the **Toolboxes** folder.   In the **Toolboxes** folder, you should see a toolbox called "Geoprocessing_and_Toolboxes.atbx".  Right click that toolbox, then hover over the "New" row and click "Script".

![Create a script tool.](resources\create_new_script_tool.png "Creating a script tool")

Creating a new script tool will open up the "Properties" wizard for this new script tool.

![New script tool properties.](resources\new_script_tool_properties.png "New script tool properties")

#### 3.  Fill in descriptive information about the tool

The first panel you'll see in the *New Script* properties is the *General* tab.  This panel contains descriptive information about the tool.  You'll need to fill out the Name, Label, and Description for this new tool.  You can choose your own wording, but we'd suggest the following

 - Name: **CountyHighwaysExporter**

 - Label: **County Highway Exporter**

 - Description: **This tool allows the user to export a single county's highways to a zipped file geodatabase for easy handoff.**

#### 4.  Enter the script logic

The only other essential part of creating a script tool is defining the execution logic.  To do this, you can click the **Execution** panel on the left of the Tool Properties wizard.  When you click **Execution** you'll see a script editing panel.  Paste the code you ran and tested in the beginning of this tutorial.

![Script logic in tool.](resources\script_logic_in_tool.png "Script logic entered into the execution tab")

Now you can run this tool. If you click the **OK** button, you will create a tool that can be accessed and run without anyone modifying the Python code.

## Modify the Script Tool

Now you've created a script tool, and it does what it's supposed to do.  It doesn't have any parameters though.  Your end users can't modify any of the inputs.  You can fix that though.

#### 1.  Edit the properties of the tool

Navigate your way back to the tool you just created.  Right-click the tool and select "Properties".

![Open the tool properties.](resources\right-click_properties.png "Open the tool properties.")

This brings you back to the "Tool Properties" window that you were working with earlier.

#### 2.  Go to the Parameters panel.

Click on the **Parameters** panel on the left of the Tool Properties window.

#### 3.  Enter paramters for the user to populate

This is where you'll define what the user enters to run your tool.  In this case, there are three parameters to fill out.  For each parameter, you'll need to specify the following:
 - **Label** - Human-readable description of what to enter
 - **Name** - Python-friendly parameter name.  The Tool Properties will create this for you automatically.
 - **Data Type** - The expected type of your input.  This can be a string, feature class, folder, or many other types.  Specifying this upfront really helps avoid errors in your script from unexpected inputs.
 - **Type** - Here you have a choice to specify whether your users have to populate this parameter.  If the parameter is "Required", your end user will not be able to run the tool without supplying a value for this parameter.  If the parameter is set to "Optional", the user can supply a value here but doesn't need to.  If the parameter is set to "Derived" then the parameter becomes an output of the tool rather than an input.  This is helpful when you're making a tool that you might want to use in ModelBuilder or a python script later.

Create the following parameters with the following settings:

 - Label: **Highways Feature Class**
   - Data Type: **Feature Class**
   - Type: **Required**
 - Label: **Output Folder**
   - Data Type: **Folder**
   - Type: **Required**
 - Label: **County**
   - Data Type: **String**
   - Type: **Required**

![setup Parameters.](resources\parameters_set_up.png "Set up Parameters.")

#### 4.  Modify the script to use the parameters

Now that you've set up the user's inputs, you'll need to specify in your script how to use the inputs.  To do this, you'll need to use an arcpy function called `GetParameterAsText()`.  This function will retreive the user inputs you set up in the previous step. You will specify which paramerter to use by providing the number of that parameter as an argument.  Go back to the **Parameters** panel if you need a refresher.

After re-writing your script tool to use the user input parameters, the body of your script should look like this.  You'll still include the defined function and package imports before this.

In [ ]:
if __name__ == "__main__":
    # input file geodatabase path
    full_fc_path = arcpy.GetParameterAsText(0)

    # output folder
    output_folder = arcpy.GetParameterAsText(1)

    # which county to export
    county = arcpy.GetParameterAsText(2)

    # export a zip file for Butte County
    zip_county_highways(full_fc_path, output_folder, county)